# Exploring the Galaxy Toys Example

We are now going to explore the Galaxy Toys example a bit more. Let's take a look at the other versions that had multiple optimal solutions, etc.

In [ ]:
# import the two Gurobi parts we need
import gurobipy as gp
from gurobipy import GRB

## Formulation 

Recall that our formulation looked like this:

| | | |
| --- | --- | --- |
| Let | | |
| $x_{s}$ | = | number of lots (dozens) of Space Rays to produce next week |
| $x_{p}$ | = | number of lots (dozens) of Phasers to produce next week |

| | | | | | | |
| --- | --- | --- | --- | --- | --- | --- |
| $\max$ | $8x_{s}$ | $+$ | $5x_{p}$ | | | |
| s.t. | $2x_{s}$ | $+$ | $1x_{p}$ | $\le$ | $1200$ | {plastic pounds} |
| | $3x_{s}$ | $+$ | $4x_{p}$ | $\le$ | $2400$ | {minutes of production} |
| | $1x_{s}$ | $+$ | $1x_{p}$ | $\le$ | $800$ | {overall production limit} |
| | $1x_{s}$ | $-$ | $1x_{p}$ | $\le$ | $450$ | {mix of products produced} |
| | $x_{s}$ | | | $\ge$ | $0$ | {non-negativity} |
| | | | $x_{p}$ | $\ge$ | $0$ | {non-negativity} |

### Range of Optimality

Suppose the objective function coefficient for space rays was really \$7. How does this change the solution? Can you answer that question using the sensitivity report? Do you have re-solve the problem witht the new data?

In [ ]:
####                ####
#    ORIGINAL MODEL    #
####                ####
# Create the model object
m = gp.Model('galaxy_toys')

# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

# Create decision variables
# We tell the solver that the variables are continuous,
#   their names, and their lower bounds
x_s = m.addVar(vtype=GRB.CONTINUOUS, name='x_s', lb=0.0)
x_p = m.addVar(vtype=GRB.CONTINUOUS, name='x_p', lb=0.0)

# Add the objective function
m.setObjective(8 * x_s + 5 * x_p)

# Add the constraints
# We can simply write out the constraints for the first parameter
# The second parameter names the constraint
m.addConstr(2*x_s + x_p <= 1200, name='plastic')
m.addConstr(3*x_s + 4*x_p <= 2400, name='labor')
m.addConstr(x_s + x_p <= 800, name='total_production')
m.addConstr(x_s - x_p <= 450, name='product_mix')

# update the model
m.update()

# solve
m.optimize()

### Getting the Results

We now want to get the results and print them in a little nicer format. 

In [ ]:
# Get the results out
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

### Sensitivity Analysis

We can easily get the reduced cost and the range of optimality for each variable. Similarly, we can extract the shadow price and the range feasibility for each constraint.

In [ ]:
# Get the reduced cost and range of optimality for each variable
for v in m.getVars():
    print(f'{v.VarName} has a reduced cost of {v.RC}')
    print(f'   and a range of optimality from {v.SAObjLow} to {v.SAObjUp}')

In [ ]:
# Get the shadow price and the range of feasibility for each constraint
for c in m.getConstrs():
    print(f'{c.constrName} has a shadow price of {c.pi}')
    print(f'   and a range of feasibility from {c.SARHSLow} to {c.SARHSUp}')

### Changing the Objective Function Coefficient for Space Rays

Let's go ahead and change the objective function coefficient from the original \\$8 to \\$7 and re-run the solver to see what we get.

In [ ]:
# Change the objective function
m.setObjective(7 * x_s + 5 * x_p)
m.update()
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

### Range of Feasibility

Suppose we had one additional pound of plastic that could be used for our production cycle. What does this additional resource do to our solution? Can you answer that with the sensitivity report?

First, let's go back to the original model. We need to reset the objective function coefficient for space rays. Then, let's change the right-hand-side of the plastic constraint and re-solve the problem.

In [ ]:
# The following method is deprecated and undocumented
# BUT it still appears to work
m.display()

In [ ]:
# One work around is to write the .lp file 
# to disk and and read it back in and 
# print it out ... not recommended for large models
m.write('./tmp.lp')
with open('./tmp.lp') as f:
    print(f.read())

In [ ]:
# Let's change objective function back to the original
m.setObjective(8 * x_s + 5 * x_p)
m.update()
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

In [ ]:
# We can get all the constraints with m.getConstrs()
for constr in m.getConstrs():
    # What does it loook like?
    print(constr)
    # What is its name?
    print(constr.ConstrName)
    # What about the RHS?
    print(constr.rhs)

In [ ]:
# We can pull out a specific constraint by its name
# Let's get plastic and store it a Python variable named plastic
plastic = m.getConstrByName('plastic')
print(plastic.rhs)

In [ ]:
# Now update the RHS with one additional pound of plastic
plastic.rhs = 1201

# Update the model and solve it
m.update()
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

### Multiple Optimal Solutions

Can we find multiple optimal solutions? We saw one case where both the reduced cost and the value of the decision variable was 0 indicating that perhaps multiple optimal solutions existed. The second case was not like that. Let's examine the second case because it seems more interesting. In this instance, space rays bring in \\$9 of profit and phasers bring in \\$4.50 of profit. We also need to go back to the original 1200 pounds of plastic.

In [ ]:
# reset the rhs of the plastic constraint to 1200
print(f'Current RHS of plastic constraint is {plastic.rhs}')

plastic.rhs = 1200

# update the model
m.update()

print(f'After changing RHS of back to original is {plastic.rhs}')

In [ ]:
# change the objective function
m.setObjective(9 * x_s + 4.5 * x_p)
m.update()
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

In [ ]:
# Get the reduced cost and range of optimality for each variable
for v in m.getVars():
    print(f'{v.VarName} has a reduced cost of {v.RC}')
    print(f'   and a range of optimality from {v.SAObjLow} to {v.SAObjUp}')

#### How to Find Other Optimal Solutions

That answer only showed a single solution. We can ask Gurobi to find multiple solutions and then look at those.

In [ ]:
# do a systematic search for the k-best solutions
m.setParam(GRB.Param.PoolSearchMode, 2)

# Update the model
m.update()

# Solve
m.optimize()

# Get the status
status = m.Status
if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('The model cannot be solved because it is infeasible or unbounded')

if status != GRB.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))

if status == GRB.OPTIMAL:
    print(f'Optimization found an OPTIMAL answer with the status of {status}')

# Print number of solutions stored
nSolutions = m.SolCount
print('Number of solutions found: ' + str(nSolutions))

#### Why Only 1 Solution?

When the variables are continuous and you have multiple optimal solutions, you will in fact have an **infinite** number of optimal solutions. Therefore, the solver will not try to search for more than one. Which one did it find?

In [ ]:
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

#### Another Possible Approach?

Because the optimal solution found above is integer, you could try converting both variables to integer instead of continuous and re-solving the updated model. When dealing with integer or binary variables, Gurobi will try to find other optimal solutions.

In [ ]:
# change the variable types to integer
for var in m.getVars():
    print(f'Variable {var.VarName} is currently of type {var.VTYPE}')
    var.setAttr(GRB.Attr.VType, GRB.INTEGER)

# update the model for changes to take effect
m.update()
print('After setting the variables to integer ...')
for var in m.getVars():
    print(f'Variable {var.VarName} is currently of type {var.VTYPE}')

In [ ]:
# Solve
m.optimize()

# Print number of solutions stored
nSolutions = m.SolCount
print(f'Number of solutions found: {nSolutions}')

In [ ]:
# Print objective value and values of the decision varaibles of solutions
for soln_num in range(m.getAttr(GRB.Attr.SolCount)):
    m.Params.SolutionNumber = soln_num
    print(f'solution {soln_num} has obj fn value of {m.PoolObjVal}')
    for v in m.getVars():
        # Need to use v.Xn to the value of the variable v for this solution
        print(f'   {v.VarName} = {v.Xn}')
    

### Success!

When using integer variables, we can get multiple optimal solutions with Gurobi. Note that within in solution you need to use `v.Xn`, where `v` is the variable, to get the values of the decision variables. In the previous version (v2) of the notebook, I had inadvertently mistyped and used `v.X` instead of the correct **`v.Xn`**. 

**&copy; 2024 - Present: Matthew D. Dean, Ph.D.   
Clinical Full Professor of Business Analytics at William \& Mary.**